# Q0: Your level of cricket experience
Honestly, 1.

# Q1: Question 1.
Determine the win records (percentage win and total wins) for each team by year and gender,
excluding ties, matches with no result, and matches decided by the DLS method in the event that, for whatever
reason, the planned innings can’t be completed. Consider only data from 2019. Which male and female teams had
the highest win percentages? Which had the highest total wins? Were these teams the same as those with the
highest win percentages? Comment on why the leaders of these two stats might differ.

In [3]:
import json
import pandas as pd
import pathlib as Path
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
def load_df_from_json(filename):
    with open(filename) as file:
        df = json.load(file)

    return pd.DataFrame(df)

def filter_odi_df(df, first_year_to_consider=1800):
    """ Remove matches with no winner or with outcome determined by D/L method. 
    Optionally filters out matches before a given year. 
    Returns only relevant columns needed to find win rate.
    """
    df['year'] = pd.to_datetime(df['dates']).dt.year
    df = df[df['year'] >= first_year_to_consider]
    df = df[(df['outcome.method'] != "D/L") & (df['outcome.winner'].notna())]
    df = df.drop_duplicates(subset=['matchid', 'year', 'teams'])
    return df[['matchid', 'gender', 'outcome.winner', 'teams', 'year']].reset_index(drop=True)

def compute_win_rate_table(df):
    win_rate = df.groupby(['teams', 'gender', 'year']).apply(
        lambda group_df: pd.Series({
            'tot_matches': len(group_df),
            'win_counts': (group_df['outcome.winner'] == group_df['teams']).sum(),
            'win_rate': (group_df['outcome.winner'] == group_df['teams']).mean()
        })
    )
    win_rate = win_rate.sort_values(by=['year', 'win_counts'], ascending=[True, False])

    assert all((win_rate['win_rate'] >= 0) & (win_rate['win_rate'] <= 1)), "Some win rate values are out of the expected range."
    assert all(win_rate['tot_matches'] >= 0), "Some total match counts are negative."
    assert all(win_rate['win_counts'] >= 0), "Some win counts are negative."

    return win_rate


def get_win_report(win_rate_df, rate=True):
    """
    Get report year by year and aggregate about team performance for male and female.
    If rate is set to false, the total wins are used instead of the rate
    """
    col = 'win_rate' if rate else 'win_counts'

    # Split the DataFrame into male and female subsets
    male_win_rate_df = win_rate_df.xs('male', level='gender')
    female_win_rate_df = win_rate_df.xs('female', level='gender')

    # Identify teams with most wins per year
    male_df_by_year = male_win_rate_df[col].groupby('year').idxmax().apply(lambda x: x[0])
    female_df_by_year = female_win_rate_df[col].groupby('year').idxmax().apply(lambda x: x[0])

    # Identify team with most wins in aggregate
    male_max_wins_agg = male_win_rate_df[col].idxmax()[0]
    female_max_wins_agg = female_win_rate_df[col].idxmax()[0]

    summary_df = pd.concat([male_df_by_year, female_df_by_year], keys=['Male', 'Female']).rename(f'{col} per year')
    display(summary_df)
    print(f"Most {col} from {first_year_to_consider} for males: {male_max_wins_agg}")
    print(f"Most {col} from {first_year_to_consider} for females: {female_max_wins_agg}")

In [ ]:
match_results_path = '../data/match_results.json'
first_year_to_consider = 2019

odi = load_df_from_json(match_results_path)
odi.head(3)

In [ ]:
odi.info()

It looks like we have two entries for each game, so we have 2153 games in total. One way to handle this would be to combine it into 1 single entry, but I'm not going to do it for now. Also, we don't need most of the columns and some values are only set when a certain event occurs (like outcome.method)

In [ ]:
# Find unique values for some interesting columns
for col in ['city', 'gender', 'match_type', 'teams', 'venue', 'outcome.method']:
    n_unique = len(odi[col].unique()) 
    print(f" Number of unique values for column {col} are: {n_unique}")

We have data about 27 diffrent teams. As expected genders are 2 (male, female) and match_type is only ODI. Let's take a look at the cities to see if they include the country.

In [ ]:
print(odi['city'].value_counts())

Unfortunately they don't. It could have been useful later to determine home and away team.

In [ ]:
odi.describe()

From this we can get an idea about total runs and wickets. Overs seem to be always 50, but as I understood they could be less if all the batters are outed before the end of the innings. Anyway I don't think we will use this info for Q1.

In [ ]:
odi_filt = filter_odi_df(odi, first_year_to_consider=first_year_to_consider)
odi_filt.head(3)

In [ ]:
odi_filt.info()

In [ ]:
odi_filt['gender'].value_counts()

In [ ]:
odi_filt['year'].value_counts()

It looks like in 2020 and 2021 there are way less games. Actually, it makes sense because COVID was there. Let's see if it's actually the case by looking at all the other years.

In [ ]:
# Let's check now all the past years to see average games
all_years_counts = filter_odi_df(odi, first_year_to_consider=1800)['year'].value_counts()
ax = all_years_counts.sort_index().plot(kind='bar', figsize=(12, 6), color='skyblue')
plt.axhline(y=all_years_counts.mean(), color='r', linestyle='--', label='Average')
plt.legend(); plt.show()

Yes, they are indeed way below average. We can also see there was a positive trend until 2019.
From now on we only consider games from 2019.

In [ ]:
# Let's create a new table to check total wins and win rate since we are gonna use it multiple times
win_rate_df = compute_win_rate_table(odi_filt)
win_rate_df.head(3)

In [ ]:
win_rate_df.info()

In [ ]:
win_rate_df.describe()

From these stats we can already see that there is a big difference in the amount of games each team has played.

In [ ]:
# Display team with most wins for each year and on aggregate
get_win_report(win_rate_df, rate=False)

In [ ]:
# Display team with best win rate for each year and on aggregate
get_win_report(win_rate_df, rate=True)

In [ ]:
# Check a specific sample to see if statistically relevant
team = "Afghanistan"
year = 2021
gender = 'male'
print(f"Total matches played by {team} {gender} team in {year}: {win_rate_df.loc[(team, gender, year), 'tot_matches']}, with a win rate of : {win_rate_df.loc[(team, gender, year), 'win_rate']}")

It would be nice to include the total games played in the report, but I'm gonna take a closer look by plotting the data. But from the Afghanistan example we can see that some teams may have 100% rates, but with very few games played.

In [ ]:
# Let's now go into more details by plotting the male data for each year
years = win_rate_df.index.get_level_values('year').unique()
fig, axs = plt.subplots(2, 3, figsize=(20, 15))
for i, year in enumerate(years):
    win_rate_df.xs((year, 'male'), level=('year', 'gender'))['win_counts'][:10].plot(kind='bar', ax=axs[0, i], title=f"Tot wins - {year}")
    win_rate_df.xs((year, 'male'), level=('year', 'gender'))['win_rate'].sort_values(ascending=False)[:11].plot(kind='bar', ax=axs[1, i], title=f"Win rate - {year}")

I think it's hard to draw conclusions from these plots alone. Let's check the aggregate for the 3 years, but let's consider the total games played as well now.

In [ ]:
# Let's check the aggregate now of most wins with total matches on the side
win_rate_df.xs('male', level='gender')[['tot_matches', 'win_counts']].groupby('teams').sum().sort_values('win_counts', ascending=False).plot(kind='bar', figsize=(12, 6), title=f"Aggregate wins from {first_year_to_consider}")

If we only consider teams with more than 10 games played, we can see that win rate and total wins start to align a bit more. Indeed Australia, India, England and especially New Zeland seem to have performed the best in the last 3 years (let's keep in mind that COVID maybe affected the results).

Overall I think that the 2 metrics are not comparable and in a context like this one are both useful. Obviously a team that has played more games has more chances to have a higher number of wins. 

In general with international games it is a bit trickier to look at stats like this because each year you may be playing different teams which can be more or less strong. In a national league with fixed teams, these metrics are definately more useful. Nevertheless even here they can give a good hint at the overall team performance.

Finally I tried to track teams performances year by year, but it's a bit challenging given the lack of data for the last 2 years. Nevertheless I think this is a good way to check for trends, especially for regular seasons. For example, if we comfirm that South Africa has played a statistically significant number of games in each 3 years, we could say that they are on a positive trend.

In [ ]:
# Assume that 'yearly_win_rate_df' is a DataFrame with teams as index, years as columns, and win rates as values.
yearly_win_rate_df = win_rate_df.xs('female', level='gender').groupby(['year', 'teams']).win_rate.mean().unstack()

plt.figure(figsize=(10, 8))
sns.heatmap(yearly_win_rate_df, cmap='YlGnBu', annot=True, fmt=".1f")
plt.title('Heatmap of Win Rates over Time')
plt.xlabel('Year')
plt.ylabel('Team')
plt.show()

# Q2.
Setting aside individual batter production, cricket teams have two main ‘resources’ for producing
runs: remaining overs and wickets. The role resources have on run production is central to the statistical method
known as ‘DLS’, which is used to award a winner in the case of incomplete/disrupted matches. Use the ball-by-ball
summaries under the innings descriptions of each men’s match to make a dataset with the run and wicket outcomes
for each delivery in a match, excluding matches with no result.
Develop a model to predict an average team’s expected runs per over. Please state or include the assump-
tions/validation used to justify your model choice. A visualization prior to modelling could be helpful to justify
your modelling decisions. Save your intermediate data with team, inning order, remaining overs, and remaining
wickets to a JSON/CSV file for Q4. Summarize your conclusions.

First I should filter out only the men data. Also let's start only with one game.

In [13]:
def filter_innings_df_with_match_df(innings, odi):
    filtered_match_ids = odi['matchid'].unique()
    assert len(filtered_match_ids) == len(odi) / 2
    innings = innings[innings['matchid'].isin(filtered_match_ids)]
    innings = innings[['matchid', 'innings', 'over', 'runs.total', 'wicket.kind', 'team']]
    innings['over_agg'] = innings['over'].apply(lambda x: int(x.split('.')[0]))
    innings = innings.drop_duplicates(subset=['matchid', 'innings', 'over', 'team'])
    return innings

def add_features_to_overs_df(overs, starting_wickets=10, n_overs=50):
    overs['wickets_cumul'] = overs.groupby(['matchid', 'innings'])['wickets'].cumsum()
    overs['runs_cumul'] = overs.groupby(['matchid', 'innings'])['runs'].cumsum()
    overs['remaining_wickets'] = starting_wickets - overs['wickets_cumul']
    overs['remaining_overs'] = n_overs - overs['over_agg']

    total_runs = overs.groupby(['matchid', 'innings'])['runs'].sum().reset_index()
    first_innings_runs = total_runs[total_runs['innings'] == 1]
    first_innings_runs = first_innings_runs.rename(columns={'runs': 'first_innings_runs'})
    first_innings_runs = first_innings_runs.drop(columns=['innings'])
    overs = overs.merge(first_innings_runs, on='matchid', how='left')
    # Set to 0 for first innings since no data is available
    overs.loc[overs['innings'] == 1, 'first_innings_runs'] = 0
    overs['runs_needed_to_par'] = overs['first_innings_runs'] - overs['runs_cumul']
    overs.loc[overs['innings'] == 1, 'runs_needed_to_par'] = 0
    overs.drop(columns=['first_innings_runs'])

    overs['last_5_overs_mean_runs'] = overs['runs'].rolling(window=5).mean()
    overs['last_5_overs_mean_runs'].fillna(2, inplace=True)
    return overs

def shift_resources(df):
    df['last_5_overs_mean_runs'] = df['last_5_overs_mean_runs'].shift(1)
    df['remaining_wickets'] = df['remaining_wickets'].shift(1)
    df['runs_needed_to_par'] = df['runs_needed_to_par'].shift(1)
    df['cumulative_runs'] = df['runs_cumul'].shift(1)

    # Initialize the first row with 10 for remaining overs and wickets, respectively
    df.loc[df.index[0], 'last_5_overs_mean_runs'] = 2
    df.loc[df.index[0], 'remaining_wickets'] = 10
    df.loc[df.index[0], 'cumulative_runs'] = 0
    df.loc[df.index[0], 'runs_needed_to_par'] = df.loc[df.index[0], 'first_innings_runs']

    return df

In [6]:
match_results_path = '../data/match_results.json'
innings_results_path = '../data/innings_results.json'

odi = load_df_from_json(match_results_path)
innings = load_df_from_json(innings_results_path)

innings.head(3)

,batsman,bowler,non_striker,runs.batsman,runs.extras,runs.total,over,team,innings,matchid,...,replacements.match.in,replacements.match.out,replacements.match.team,replacements.match.reason,replacements.match.in.1,replacements.match.out.1,replacements.match.reason.1,replacements.match.team.1,extras.penalty,extras.wides
0,DA Warner,Mohammad Amir,TM Head,0,0,0,0.1,Australia,1,1000887,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DA Warner,Mohammad Amir,TM Head,0,0,0,0.2,Australia,1,1000887,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DA Warner,Mohammad Amir,TM Head,0,0,0,0.3,Australia,1,1000887,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
innings.info()

In [ ]:
# Find unique values for some interesting columns
for col in ['batsman', 'bowler', 'non_striker', 'team', 'wicket.kind']:
    n_unique = len(innings[col].unique()) 
    print(f" Number of unique values for column {col} are: {n_unique}")

In [ ]:
print(innings['wicket.kind'].value_counts())

In [ ]:
innings.describe()

In [ ]:
odi_filt = filter_odi_df(odi, first_year_to_consider=1800)
innings_filt = filter_innings_df_with_match_df(innings, odi_filt)
innings_filt.tail(55)

In [ ]:
innings_filt.info()

In [ ]:
innings_filt.describe()

In [8]:
overs = innings_filt.groupby(['matchid', 'innings', 'over_agg', 'team']).agg(
    runs=('runs.total', 'sum'),
    wickets=('wicket.kind', 'count')
).reset_index()

overs.tail(5)

,matchid,innings,over_agg,team,runs,wickets
168673,997995,2,43,Scotland,4,0
168674,997995,2,44,Scotland,2,1
168675,997995,2,45,Scotland,9,0
168676,997995,2,46,Scotland,7,0
168677,997995,2,47,Scotland,4,0


In [14]:
overs_ds = add_features_to_overs_df(overs)
# Shift features so we don't leak the future into the data
overs_ds = overs_ds.groupby(['matchid', 'innings']).apply(shift_resources)
# Unpack groups to process it more easily
overs_ds = overs_ds.drop(columns=["matchid", "innings"]).reset_index()
overs_ds.head(5)

,matchid,innings,level_2,over_agg,team,runs,wickets,wickets_cumul,runs_cumul,remaining_wickets,remaining_overs,first_innings_runs,runs_needed_to_par,last_5_overs_mean_runs,cumulative_runs
0,1000887,1,0,0,Australia,1,0,0,1,10.0,50,0,0.0,2.0,0.0
1,1000887,1,1,1,Australia,1,0,0,2,10.0,49,0,0.0,2.0,1.0
2,1000887,1,2,2,Australia,3,0,0,5,10.0,48,0,0.0,2.0,2.0
3,1000887,1,3,3,Australia,6,0,0,11,10.0,47,0,0.0,2.0,5.0
4,1000887,1,4,4,Australia,2,2,2,13,10.0,46,0,0.0,2.0,11.0


In [47]:
overs_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168678 entries, 0 to 168677
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   matchid                 168678 non-null  object 
 1   innings                 168678 non-null  int64  
 2   level_2                 168678 non-null  int64  
 3   over_agg                168678 non-null  int64  
 4   team                    168678 non-null  object 
 5   runs                    168678 non-null  int64  
 6   wickets                 168678 non-null  int64  
 7   wickets_cumul           168678 non-null  int64  
 8   runs_cumul              168678 non-null  int64  
 9   remaining_wickets       168678 non-null  float64
 10  remaining_overs         168678 non-null  int64  
 11  first_innings_runs      168678 non-null  int64  
 12  runs_needed_to_par      168678 non-null  float64
 13  last_5_overs_mean_runs  168678 non-null  float64
 14  cumulative_runs     

In [ ]:
overs_ds.describe()

In [46]:
# This will return all the rows where cumulative wickets are greater than 10
anomalies = overs_ds[overs_ds['runs_needed_to_par'] < 0]
print(len(anomalies))
display(anomalies)
# anomaly_details = overs_ds[(overs_ds['matchid'].isin(anomalies['matchid'])) & 
#                              (overs['innings'].isin(anomalies['innings']))]

0


,matchid,innings,level_2,over_agg,team,runs,wickets,wickets_cumul,runs_cumul,remaining_wickets,remaining_overs,first_innings_runs,runs_needed_to_par,last_5_overs_mean_runs,cumulative_runs


In [45]:
# Since it's only one case we can just modify it with a reasonable value (would need better handling though)
overs_ds[overs_ds['runs_needed_to_par'] < 0] = 3 

In [ ]:
# Sanity checks
assert overs_ds['runs'].between(0, 36).all()
assert overs_ds['remaining_wickets'].between(0, 10).all()
assert overs_ds['remaining_overs'].between(0, 50).all()
assert overs_ds['wickets'].between(0,6).all()
assert overs_ds['runs_needed_to_par'].between(0,1000*50*6*6).all()
assert overs_ds.isnull().sum().all() == 0
assert overs_ds.duplicated().sum() == 0

In [ ]:
overs[[
                    "remaining_overs",
                    "remaining_wickets",
                    "innings",
                    "cumulative_runs",
                    "runs_needed_to_par",
                    "last_5_overs_mean_runs",
]].to_csv('../data/intermediate_data.csv', index=False)

In [ ]:
# Import necessary libraries
import json
import pandas as pd
import pathlib as Path
import os
import numpy as np

# Load the match results data
with open('../data/match_results.json') as file:
    match_results = json.load(file)

match_results_df = pd.DataFrame(match_results)

# Rename 'teams' column in match_results_df to 'team' for consistency
match_results_df = match_results_df.rename(columns={'teams': 'team'})
match_results_df = match_results_df[match_results_df['outcome.method'] != "D/L"]

# Load the ball-by-ball innings data
with open('../data/innings_results.json') as file:
    innings_data = json.load(file)

innings_df = pd.DataFrame(innings_data)

# Remove matches with no result
match_results_df = match_results_df[match_results_df['outcome.winner'].notna()]

# Get matchids of matches with result
match_ids_with_result = match_results_df['matchid'].unique()

# Filter innings_df to include only matchids of matches with result
innings_df = innings_df[innings_df['matchid'].isin(match_ids_with_result)]

# Merge the gender field from match_results_df to innings_df
innings_df = innings_df.merge(match_results_df[['matchid', 'team', 'gender', 'dates']], on=['matchid', 'team'], how='left')

# Filter for male matches
innings_df = innings_df[innings_df['gender'] == 'male']

# Convert the 'dates' column to datetime format
innings_df['dates'] = pd.to_datetime(innings_df['dates'])

# Filter out matches before 2019
# innings_df = innings_df[innings_df['dates'].dt.year == 2020]

# Keep only necessary columns 
innings_df = innings_df[['matchid', 'innings', 'over', 'runs.total', 'wicket.kind', 'team']]
innings_df = innings_df.drop_duplicates(subset=['matchid', 'innings', 'over', 'team'])
# innings_df = innings_df[innings_df['innings'] == 1]

# Compute wickets and cumulative wickets directly in innings_df
innings_df['wickets'] = innings_df['wicket.kind'].notnull()
innings_df['wickets_cumul'] = innings_df.groupby(['matchid', 'innings', 'team'])['wickets'].cumsum()
innings_df['runs_cumul'] = innings_df.groupby(['matchid', 'innings', 'team'])['runs.total'].cumsum()
assert innings_df['wickets_cumul'].between(0,10).all()
# Compute remaining wickets and remaining overs
innings_df['over'] = innings_df['over'].apply(lambda x: int(x.split('.')[0]) if int(x.split('.')[1]) == 6 else float(x)).astype(int)
innings_df['remaining_wickets'] = 10 - innings_df['wickets_cumul']
innings_df['remaining_overs'] = 50 - innings_df['over']

# Calculate total runs per match per innings
total_runs = innings_df.groupby(['matchid', 'innings'])['runs.total'].sum().reset_index()

# Filter to keep only the first innings
first_innings_runs = total_runs[total_runs['innings'] == 1]

# Rename columns for merging
first_innings_runs = first_innings_runs.rename(columns={'runs.total': 'first_innings_runs'})
first_innings_runs = first_innings_runs.drop(columns=['innings'])

# Merge first_innings_runs into the main dataframe
innings_df = innings_df.merge(first_innings_runs, on='matchid', how='left')

# Forward fill the NaNs to propagate the first_innings_runs to the second innings
innings_df['first_innings_runs'] = innings_df.groupby('matchid')['first_innings_runs'].ffill()

# At this point, the 'first_innings_runs' column for the first innings will be its own score, 
# which is not what we want. We set 'first_innings_runs' to NaN for the first innings:
print(innings_df.columns)
innings_df.loc[innings_df['innings'] == 1, 'first_innings_runs'] = 0
innings_df['runs_needed_to_par'] = innings_df['first_innings_runs'] - innings_df['runs_cumul']
innings_df.loc[innings_df['innings'] == 1, 'runs_needed_to_par'] = 0
#  Display the dataframe
# with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.precision', 3):
#     display(innings_df)

grouped_df = innings_df.groupby(['matchid', 'innings', 'team', 'over']).agg(
    runs=('runs.total', 'sum'),
    wickets=('wickets', 'sum'),  
    remaining_wickets=('remaining_wickets', 'min'),  # minimum remaining wickets in the over
    remaining_overs=('remaining_overs', 'min'),  # minimum remaining overs in the over
    cumulative_runs=('runs_cumul', 'max'),  # minimum remaining overs in the over
    first_innings_runs=('first_innings_runs', 'max'),
    runs_needed_to_par=('runs_needed_to_par', 'min')
).reset_index()
grouped_df = grouped_df.groupby(['matchid', 'innings']).apply(shift_resources)


In [ ]:
grouped_df[[
                    "remaining_overs",
                    "remaining_wickets",
                    "innings",
                    "cumulative_runs",
                    "runs_needed_to_par",
                    "last_5_overs_mean_runs",
]].to_csv('../data/intermediate_data.csv', index=False)

In [ ]:
# Display the dataframe
# with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.precision', 3):
#     display(grouped_df)

In [ ]:
# Display the dataframe
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.precision', 3):
    display(grouped_df[grouped_df['matchid']=="1153846"])

### Sanity Checks

In [ ]:
# This will return all the rows where cumulative wickets are greater than 10
anomalies = grouped_df[grouped_df['runs_needed_to_par'] < 0]
grouped_df[grouped_df['runs_needed_to_par'] < 0] = 3
print(len(grouped_df))
# Display these instances
print(len(anomalies))
display(anomalies)
anomaly_details = innings_df[(innings_df['matchid'].isin(anomalies['matchid'])) & 
                             (innings_df['innings'].isin(anomalies['innings']))]
print(anomaly_details)

In [ ]:
# Sanity checks
assert grouped_df['runs'].between(0, 36).all()
assert grouped_df['remaining_wickets'].between(0, 10).all()
assert grouped_df['remaining_overs'].between(0, 50).all()
assert innings_df['wickets'].between(0,6).all()
print(min(grouped_df['runs_needed_to_par']), max(grouped_df['runs_needed_to_par']))
assert grouped_df['runs_needed_to_par'].between(0,1000*50*6*6).all()
assert grouped_df.isnull().sum().all() == 0
assert grouped_df.duplicated().sum() == 0
# groupd_df = grouped_df[['over', 'remaining_overs', 'remaining_wickets', 'runs']]
# assert (grouped_df.sort_values(['matchid', 'innings', 'over'])['remaining_overs'].diff().dropna() <= 0).all()

In [48]:
grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 148436 entries, ('1000887', 1, 0) to ('997995', 2, 148435)
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   matchid                 148436 non-null  object 
 1   innings                 148436 non-null  int64  
 2   team                    148436 non-null  object 
 3   over                    148436 non-null  int32  
 4   runs                    148436 non-null  int64  
 5   wickets                 148436 non-null  int64  
 6   remaining_wickets       148436 non-null  float64
 7   remaining_overs         148436 non-null  int32  
 8   cumulative_runs         148436 non-null  float64
 9   first_innings_runs      148436 non-null  int64  
 10  runs_needed_to_par      148436 non-null  float64
 11  last_5_overs_mean_runs  148436 non-null  float64
dtypes: float64(4), int32(2), int64(4), object(2)
memory usage: 18.7+ MB


In [ ]:
grouped_df.describe()

### Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the percentage of remaining resources
grouped_df['percentage_of_combined_resources'] = (grouped_df['remaining_wickets'] / 10 + grouped_df['remaining_overs'] / 50) / 2 * 100

plt.figure(figsize=(10, 8))

# Create a line plot for each value of remaining wickets
for wickets in grouped_df['remaining_wickets'].unique():
    subset_df = grouped_df[grouped_df['remaining_wickets'] == wickets]
    sns.lineplot(x='remaining_overs', y='percentage_of_combined_resources', data=subset_df, label=f'Remaining wickets: {wickets}')

plt.legend()
plt.xlabel('Remaining Overs')
plt.ylabel('Percentage of Combined Resources')
plt.title('Remaining Resources vs Remaining Overs for Different Numbers of Wickets')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Remaining overs vs Runs
plt.figure(figsize=(8, 6))
sns.scatterplot(data=grouped_df, x="remaining_overs", y="runs")
plt.title('Remaining Overs vs Runs')
plt.xlabel('Remaining Overs')
plt.ylabel('Runs')
plt.show()

# 2. Remaining wickets vs Runs
plt.figure(figsize=(8, 6))
sns.scatterplot(data=grouped_df, x="remaining_wickets", y="runs")
plt.title('Remaining Wickets vs Runs')
plt.xlabel('Remaining Wickets')
plt.ylabel('Runs')
plt.show()

# 3. Remaining overs and Remaining wickets vs Runs
plt.figure(figsize=(8, 6))
sns.scatterplot(data=grouped_df, x="remaining_overs", y="runs", hue="remaining_wickets", palette="viridis")
plt.title('Remaining Overs and Remaining Wickets vs Runs')
plt.xlabel('Remaining Overs')
plt.ylabel('Runs')
plt.legend(title='Remaining Wickets')
plt.show()

# 4. Percentage of combined resources vs Runs
plt.figure(figsize=(8, 6))
sns.scatterplot(data=grouped_df, x="percentage_of_combined_resources", y="runs")
plt.title('Percentage of Combined Resources vs Runs')
plt.xlabel('Percentage of Combined Resources')
plt.ylabel('Runs')
plt.show()


In [ ]:
# Line plots

# 1. Remaining overs vs Runs
plt.figure(figsize=(8, 6))
sns.lineplot(data=grouped_df, x="remaining_overs", y="runs")
plt.title('Remaining Overs vs Runs')
plt.xlabel('Remaining Overs')
plt.ylabel('Runs')
plt.show()

# 2. Remaining wickets vs Runs
plt.figure(figsize=(8, 6))
sns.lineplot(data=grouped_df, x="remaining_wickets", y="runs")
plt.title('Remaining Wickets vs Runs')
plt.xlabel('Remaining Wickets')
plt.ylabel('Runs')
plt.show()

# 3. Remaining overs and Remaining wickets vs Runs
# For this, we will make a heatmap
pivot_table = grouped_df.pivot_table(index='remaining_overs', columns='remaining_wickets', values='runs', aggfunc='mean')
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_table, cmap='viridis')
plt.title('Remaining Overs and Remaining Wickets vs Runs')
plt.xlabel('Remaining Wickets')
plt.ylabel('Remaining Overs')
plt.show()

# 4. Percentage of combined resources vs Runs
plt.figure(figsize=(8, 6))
sns.lineplot(data=grouped_df, x="percentage_of_combined_resources", y="runs")
plt.title('Percentage of Combined Resources vs Runs')
plt.xlabel('Percentage of Combined Resources')
plt.ylabel('Runs')
plt.show()


In [ ]:
#  1. Line plots
plt.figure(figsize=(8, 6))
sns.lineplot(data=grouped_df, x="remaining_overs", y="runs")
plt.title('Remaining Overs vs Runs')
plt.xlabel('Remaining Overs')
plt.ylabel('Runs')
plt.show()

# 1. Line plots
plt.figure(figsize=(8, 6))
sns.lineplot(data=grouped_df, x="remaining_overs", y="remaining_wickets")
plt.title('Remaining Overs vs remaining wickets')
plt.xlabel('Remaining Overs')
plt.ylabel('wickets')
plt.show()

# 2. Box plots
plt.figure(figsize=(8, 6))
sns.boxplot(data=grouped_df, x="remaining_overs", y="runs")
plt.title('Remaining Overs vs Runs')
plt.xlabel('Remaining Overs')
plt.ylabel('Runs')
plt.show()

# 3. Histograms
plt.figure(figsize=(8, 6))
sns.histplot(data=grouped_df, x="runs")
plt.title('Distribution of Runs')
plt.xlabel('Runs')
plt.ylabel('Count')
plt.show()

# 3. Histograms
plt.figure(figsize=(8, 6))
sns.histplot(data=grouped_df, x="wickets")
plt.title('Distribution of wickets')
plt.xlabel('Wickets')
plt.ylabel('Count')
plt.show()

# 3. Histograms
plt.figure(figsize=(8, 6))
sns.histplot(data=grouped_df, x="remaining_wickets")
plt.title('Distribution of remaining wickets')
plt.xlabel('remaining wickets')
plt.ylabel('Count')
plt.show()

# 3. Histograms
plt.figure(figsize=(8, 6))
sns.histplot(data=grouped_df, x="remaining_overs")
plt.title('Distribution of remaining overs')
plt.xlabel('remaining overs')
plt.ylabel('Count')
plt.show()

# 4. Heatmaps
plt.figure(figsize=(8, 6))
heatmap_data = grouped_df.pivot_table(index='remaining_wickets', columns='remaining_overs', values='runs', aggfunc='mean')
sns.heatmap(heatmap_data, cmap="YlGnBu")
plt.title('Runs by Remaining Overs and Wickets')
plt.xlabel('Remaining Overs')
plt.ylabel('Remaining Wickets')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Histogram for runs
plt.figure(figsize=(8, 6))
sns.histplot(data=grouped_df, x="runs", bins=30, kde=True)
plt.title('Distribution of Runs')
plt.xlabel('Runs')
plt.ylabel('Frequency')
plt.show()

# Histogram for remaining overs
plt.figure(figsize=(8, 6))
sns.histplot(data=grouped_df, x="remaining_overs", bins=30, kde=True)
plt.title('Distribution of Remaining Overs')
plt.xlabel('Remaining Overs')
plt.ylabel('Frequency')
plt.show()

# Histogram for remaining wickets
plt.figure(figsize=(8, 6))
sns.histplot(data=grouped_df, x="remaining_wickets", bins=10, kde=True)
plt.title('Distribution of Remaining Wickets')
plt.xlabel('Remaining Wickets')
plt.ylabel('Frequency')
plt.show()


### Model training

In [ ]:
np.mean(abs(np.array(y_pred - y * grouped_df["runs"].max())))

In [ ]:
X /= np.array([10, 50])
print(max(X[:,0]))
print(max(X[:,1]))

In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Prepare the data
X = np.sqrt(np.array(overs_ds[["remaining_overs", "remaining_wickets", "innings", "cumulative_runs", "runs_needed_to_par", "last_5_overs_mean_runs"]]))
# X = np.sqrt(np.array(grouped_df[["remaining_overs"]]))
y = np.array(overs_ds["runs"])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing steps
preprocessor = StandardScaler()

# Dictionary of models
models = {
    "Linear Regression": LinearRegression(),
    "XGBoost": XGBRegressor(objective ='reg:squarederror', n_estimators=100, max_depth=7, eta=1, subsample=0.7, colsample_bytree=0.8),
    "Random Forest": RandomForestRegressor(n_estimators=50, random_state=42),
    "MLP": MLPRegressor(hidden_layer_sizes=(50, 10), learning_rate_init=0.01, max_iter=1000, random_state=1)
}

# Loop through the models and train and evaluate each one
for model_name, model in models.items():
    # Create the pipeline: preprocessor + model
    # pipeline = Pipeline(steps=[('preprocessor', preprocessor),
    #                        ('model', model)])   # Train the model
    model.fit(X_train, y_train)
    # Make predictions on the train set
    y_pred_train = model.predict(X_train)
    
    # Make predictions on the test set
    y_pred_val = model.predict(X_test)
    # Evaluate the model
    mse_train = mean_squared_error(y_train, y_pred_train) 
    mse_test = mean_squared_error(y_test, y_pred_val)
    r2 = r2_score(y_test, y_pred_val)
    print(f"Model: {model_name}")
    print(f"RMSE train: {np.sqrt(mse_train)}")
    print(f"RMSE test: {np.sqrt(mse_test)}")
    print(f"R2 score test: {r2}")
    print("--------------------------")


Model: Linear Regression
RMSE train: 3.359581080332122
RMSE test: 3.3309633799330873
R2 score test: 0.10468450969210452
--------------------------
Model: XGBoost
RMSE train: 3.073399112840215
RMSE test: 3.6254357939570765
R2 score test: -0.06061268902084427
--------------------------
Model: Random Forest
RMSE train: 1.6206600306266035
RMSE test: 3.5272099269203823
R2 score test: -0.003919750548120993
--------------------------
Model: MLP
RMSE train: 3.317614470237227
RMSE test: 3.2959531152875208
R2 score test: 0.12340612314129107
--------------------------


In [ ]:
from joblib import dump
dump(models["MLP"], "../models/cricket_model.pkl")

# Q3. 
More generally and unrelated to cricket or the previous questions, model deployment in a production
environment is an important aspect of an engineer’s toolkit. Describe a scalable architecture (a diagram may
be helpful) that would be appropriate for deploying a model that predicts frame-level play values into a cloud
environment with the following assumptions:
• Spatial temporal high frame-rate data (~1 GB per game)
• Play-values are predicted at each frame of a game
• Delivery of game predictions are expected to be delivered overnight
• 500 games per season with 50 games a day
• 5 seasons of existing data
• Model training resources:
– 8 hour runtime with multiple cores (8) and large memory usage
• Model prediction resources:
– 60 min runtime per game with a single CPU and 4 GB of memory usage
List out the services, tooling, and reasoning for the choices of architecture. For example, a LAMP stack could be
appropriate for an internal home network webpage on a Raspberry Pi.

Please refer to the diagram below.

Although the question is focused on deployment, I decided to include development as well because I think that you can only have a solid production environment if you set up the dev one properly. 

I propose to use AWS services because they are the ones I'm most familiar with, but any cloud provider would do. Also for simplicity I'm not going to delve into security details.

Overall, I would use Kubeflow to orchestrate tasks, keep track of experiments/models and serve in production. Kubeflow is a workflow management system optimized for machine learning. It runs on kubernetes and it can be integrated into the AWS environment with EKS.

First of all, I would make two accounts, one for development and one for production, so that it is easier to separate costs and also makes it harder for developers to accidentaly modify production setup by limiting its access. Depending on the people available in the team, we could set up also a staging account (with the same architecture as the prod environment), for testing before real deployment.

We can store previous seasons data on a S3 bucket. Since the total data available is around 2.5 TB, and we don't expect exponential increase in the near future, this will suffice. If needed, we can also add an RDS instance to store structured data like game metadata.

Developers can access the data with containerized Jupyter notebooks, ensuring high reproducibility. Each experiment is tracked with kubeflow (integration of other tools like w&b also possible) and the models + training artifacts are stored in a S3 bucket.

When the developers set a new model as baseline, they can define a Kubeflow pipeline to automate model training. This includes retrieving the data, preprocessing and doing the actual training on a AWS machine with GPU. Each step of the pipeline runs in a container that a developer previously pushed to a container registry. This approach ensures reproducibility.

On the production side, we also have a Kubeflow cluster. As soon as the game data is ready (this could be triggered by an API call or an event) a lambda function is run and it launches the inference of the model.  Ideally, the data is already available on an S3 bucket in the production account. We can then store results and metadata in an RDS SQL database and potentially transfer the data to the dev account for future retraining. By using Kubernetes, we can adjust the number of active nodes based on the number of concurrent requests, making this architecture inherently scalable.

Registration of containers to the registry as well as model deployment and testing can be automated with CI/CD linked to pull requests in the git repository.

In terms of costs, for the storage of past games we would spend either 405$ or 121$ depending on the urgency of retrainings (instant retrieval for the former and up to 12hrs for the latter). Given that we can shut down nodes when we don't use time, inference time in a year is 500 hours. We can further reduce costs by choosing spot instances since we don't need to operate in real time. We can then choose an instance like a m6g.medium, with 4GiB of memory and a single core. This will give a yearly cost of 17.75$. 
For training the model the choice will depend if we need access to machines with GPUs. Since we are dealing with videos, this is very likely. We can then choose a g3s.xlarge with 8Gb of vRAM and 4 CPUs, coming at a hourly cost of 0.225$ for spot intances. If we consider 1 retraining per week during the season (~32), the cost is 32x8x0.225=57.6$

Summing it up, considering periodic retraining, the main fixed cost for this architecture is around 195$ per year.

In [ ]:
from IPython.display import Image, display
display(Image(filename='../images/architecture-diagram.png'))

# Q4
Question 4. Save your model from Q2 into a file and create a packaged solution for being able to build, deploy
and run your model locally. We are expecting a solution where local runs can be initiated from the command line,
not an API-style deployment. As a way to test your package, create a shell script that takes data saved from Q2,
filters for the first 5 Ireland overs, sends them to your model, and displays the model results to stdout.

Instructions to run the code.